In [ ]:
from PIL import Image
import numpy as np
import sigmf
import matplotlib.pyplot as plt
import swiftfox # only used for plotting
from scipy.signal import resample_poly, hilbert, spectrogram
from matplotlib import cm
from scipy.io import wavfile

# Load Wanted Poster
* Throw away last row of poster
* Insert Preamble row with 16 sync symbols and 7 primes

In [ ]:
with Image.open('2022-09_gort-wanted_v1_dither-edd-stucki.png') as img:
    print(np.array(img.convert('1')))

In [ ]:
preamble = [255, 0] * 16
for run in [2,3,5,7,11,13,17]:
    preamble += [255]*run + [0]
preamble = np.array(preamble, dtype=np.uint8)

with Image.open('2022-09_gort-wanted_v1_dither-edd-stucki.png') as img:
    # display(img)
    # display(img.convert('1'))
    npimg = np.array(img.convert('L'))
    # npimg = np.array(img.convert('1'))
    # place prime sequence at start
    npimg = np.vstack((preamble, npimg[0:-1]))
    print(npimg.shape)
    img = Image.fromarray(npimg)
    vimg = Image.fromarray(np.uint8(cm.viridis(npimg)*255))
    display(img.convert('1'))
    npimg2 = np.array(img.convert('1'))
    display(img)
    display(vimg)
# print(npimg[0:3])
# with open('1bitimg_unpacked.b','wb') as derp:
#     derp.write(npimg2.tobytes())
with open('1bitimg_packed.b', 'wb') as derp:
    # 5 bit preamble to remove later
    derp.write(np.packbits(np.hstack((
        np.ones(5, dtype=bool),
        npimg2.ravel(),
        np.zeros(16, dtype=bool),
    ))).tobytes())
    

In [ ]:
# assure I have an even number of bytes
(5+npimg2.ravel().shape[0])/8

In [ ]:
# assure I have boolean array of correct shape
npimg2.ravel()

## Load FSK Symbols output from flowgraph

In [ ]:
# wierd padding on GFSK mod.
# start: inserted 5, cut off 7
# end: inserted 16, cut off 14
samp_rate = 400e3
sps = 777
samps_fsk = np.fromfile('fsk_mod.fc32', dtype=np.complex64)[7*sps:-14*sps]

In [ ]:
107*97, samps_fsk.shape[0]/400e3

In [ ]:
107*97*777/400e3

In [ ]:
npimg2.ravel()[-8:], npimg2.shape

 Inspect FSK Bits

In [ ]:
# clip = resample_poly(samps_fsk[sps*(107*2-3):sps*107*2],1,20)
clip = resample_poly(samps_fsk[sps*0:sps*107],1,10)
# clip = resample_poly(samps_fsk[-sps*(8):],1,10)
plt.plot(np.unwrap(np.angle(clip)))
for i in range(0,5):
    plt.axvline(i*sps*8//10,alpha=.8,color='red')
plt.title('first 32 bits should be idle')
plt.show()
# plt.plot(clip.real)
# plt.plot(clip.imag)
# plt.show()
# plt.figure(figsize=(16,5))
# swiftfox.smartspec(clip, samp_rate=400e3/10,detail=.2)
# plt.show()

### pad for 60 second capture (rev2)

In [ ]:
pad_required = int(60 * samp_rate - samps_fsk.shape[0])
pad_front = int(13.171 * samp_rate)
print(pad_required/400e3)
samps_fsk_padded = np.hstack((
    np.zeros(pad_front,dtype=np.complex64),
    samps_fsk,
    np.zeros(pad_required-pad_front,dtype=np.complex64)))
samps_fsk_padded.shape[0]/400e3

# read background & insert our data
Note raw waveform not in repo

In [ ]:
handle = sigmf.sigmffile.fromfile('/home/kal29868/Projects/gr-k6of/arc0/out.sigmf-data')
samps = handle.read_samples()[0:int(60*samp_rate)]

In [ ]:
len(handle)/400e3

In [ ]:
# the built-in scipy spectrogram sucks
# plt.figure(figsize=(16,9))
# spec_freqs, spec_times, spec = spectrogram(samps[0:int(samp_rate*10)], samp_rate, nperseg=8192)
# plt.imshow(spec,aspect='auto', )
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

In [ ]:
plt.figure(figsize=(16,9))
_=swiftfox.smartspec(samps, samp_rate=samp_rate)
plt.show()

### Insert our data
+79.06 KHz offset puts the signal at Hydrogen Line / 100

In [ ]:
# lower the amplitude of our inserted signal so it looks okay against the background
shifted = swiftfox.freqshift((samps_fsk_padded / 20e3), samp_rate, 79.06e3+100e3).astype(np.complex64)
merged = shifted + samps

In [ ]:
plt.figure(figsize=(16,9))
_=swiftfox.summary(merged, samp_rate=samp_rate)

## add newspaper instructions WAV as AM-USB-SC

In [ ]:
# Insert WAV
#!ffmpeg -y -i /home/kal29868/Dropbox/shared/grcon/newspaper.mp4 -af aresample=resampler=soxr -ar 48000 newspaper.wav

In [ ]:
news_rate, newspaper = wavfile.read('newspaper_truncated.wav')
hil_newspaper = hilbert(newspaper.astype(np.float32)).astype(np.complex64) # now we have SSB-SC (USB) version
len(newspaper)/48e3


In [ ]:
plt.plot(hil_newspaper[-5000:]);plt.show()

In [ ]:
_=swiftfox.smartspec(hil_newspaper, samp_rate=48e3)

In [ ]:
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt


def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Turns out we don't need to low pass filter since the audio source is already filtered
# news_filt = butter_lowpass_filter(newspaper, 3.5e3, fs=48e3)
news_upsampled = resample_poly(hil_newspaper, 25, 3).astype(np.complex64) # resample to 400KHz
swiftfox.smartspec(news_upsampled, samp_rate=samp_rate)
plt.show()

In [ ]:
plt.plot(hil_newspaper);plt.show()

In [ ]:
pad_required = int(60*samp_rate - news_upsampled.shape[0])
pad_front = int(8 * samp_rate)
print(pad_required/400e3)
samps_news_padded = np.hstack((
    np.zeros(pad_front, dtype=np.complex64),
    news_upsampled,
    np.zeros(pad_required-pad_front, dtype=np.complex64)))
samps_news_padded.shape[0]/400e3

In [ ]:
shifted = swiftfox.freqshift((samps_news_padded / 1e6/10), samp_rate, -65e3).astype(np.complex64) #-65k
merged2 = merged + shifted
# merged2[pad_front:pad_front+len(shifted)] += shifted
# merged2 = merged2.astype(np.complex64)
plt.figure(figsize=(16,9))
swiftfox.summary(merged2, samp_rate=samp_rate)
plt.show()

### Add Cat Piano

In [ ]:
_, cat_piano = wavfile.read('cat_piano.wav')
cat_piano = cat_piano.T[0] # Left Channel Only
hil_cat = hilbert(cat_piano.astype(np.float32)).astype(np.complex64) # now we have SSB-SC (USB) version
len(hil_cat)/48e3

In [ ]:
hil_cat.shape

In [ ]:
cat_upsampled = resample_poly(hil_cat, 25, 3).astype(np.complex64)
pad_required = int(60*samp_rate - cat_upsampled.shape[0])
pad_front = int(4 * samp_rate)
print(pad_required/400e3)
samps_cat_padded = np.hstack((
    np.zeros(pad_front, dtype=np.complex64),
    cat_upsampled,
    np.zeros(pad_required-pad_front, dtype=np.complex64)))
samps_cat_padded.shape[0]/400e3

In [ ]:
shifted = swiftfox.freqshift((samps_cat_padded / 1e6/50), samp_rate, 8.5e3).astype(np.complex64)
merged3 = merged2 + shifted
# merged2[pad_front:pad_front+len(shifted)] += shifted
# merged2 = merged2.astype(np.complex64)
plt.figure(figsize=(16,9))
swiftfox.summary(merged3, samp_rate=samp_rate)
plt.show()

### Add Cat Piano

In [ ]:
_, cat_piano = wavfile.read('cat_piano.wav')
cat_piano = cat_piano.T[0] # left channel only
cat_piano_ssb = hilbert(cat_piano.astype(np.float32)).astype(np.complex64) # now we have SSB-SC (USB) version

In [ ]:
cat_upsampled = resample_poly(cat_piano_ssb, 25, 3).astype(np.complex64) # resample to 400KHz

In [ ]:
pad_required = int(60 * samp_rate - cat_upsampled.shape[0])
pad_front = int(4 * samp_rate)
print(pad_required/400e3)
cat_padded = np.hstack((
    np.zeros(pad_front,dtype=np.complex64),
    cat_upsampled,
    np.zeros(pad_required-pad_front,dtype=np.complex64)))
cat_padded.shape[0]/400e3

## Write SigMF Archive

In [ ]:
import datetime as dt
import numpy as np
import sigmf
from sigmf import SigMFFile
from sigmf.utils import get_data_type_str

# write those samples to file in cf32_le
merged3.tofile('/tmp/wanted.sigmf-data')

# create the metadata
meta = SigMFFile(
    data_file='/tmp/wanted.sigmf-data', # extension is optional
    global_info = {
        SigMFFile.DATATYPE_KEY: get_data_type_str(merged3),  # in this case, 'cf32_le'
        SigMFFile.SAMPLE_RATE_KEY: samp_rate,
        SigMFFile.AUTHOR_KEY: 'Kyle Logue (K6OF)',
        SigMFFile.DESCRIPTION_KEY: 'GrCon2022 Aero part1: wanted',
        SigMFFile.VERSION_KEY: sigmf.__version__,
        SigMFFile.LICENSE_KEY: "https://creativecommons.org/licenses/by-sa/4.0/",
    }
)

# create a capture key at time index 0
meta.add_capture(0, metadata={
    SigMFFile.FREQUENCY_KEY: 14125000.010244548,
    SigMFFile.DATETIME_KEY: "2022-08-28T23:25:15.111497000Z",
})

meta.add_annotation(int(4*samp_rate), len(cat_upsampled), metadata = {
    SigMFFile.FLO_KEY: 14125000-8.5e3,#-14125000,
    SigMFFile.FHI_KEY: 14125000-8.5e3+4e3,#-14125000,
    SigMFFile.COMMENT_KEY: 'cat piano',
})

# add an annotation at sample 100 with length 200 & 10 KHz width
meta.add_annotation(int(8*samp_rate), len(news_upsampled), metadata = {
    SigMFFile.FLO_KEY: 14190000.0,#-14125000,
    SigMFFile.FHI_KEY: 14190000.0+3.5e3,#-14125000,
    SigMFFile.COMMENT_KEY: 'a clue',
})

# check for mistakes & write to disk
assert meta.validate()
meta.archive('wanted.sigmf')
_=swiftfox.summary(merged3, samp_rate=400e3, sigmf_predict=meta._metadata)

## Test Decode Wanted Poster

In [ ]:
shifted = swiftfox.freqshift(merged3, samp_rate, -79.06e3-100e3).astype(np.complex64)
butterup = butter_lowpass_filter(shifted, 10e3, samp_rate)[int(samp_rate*12):int(samp_rate*35)]

In [ ]:
_=swiftfox.summary(butterup,samp_rate=samp_rate)

In [ ]:
indent = 468000+370
butterup2 = butterup[indent:indent+107*97*777]
phase = np.unwrap(np.angle(butterup2))

In [ ]:
plt.figure(figsize=(16,9))
# plt.plot(butterup2[7500000:], lw=.1)
for idx in range(10):
    plt.axvline(idx*777)
plt.plot(phase[0:10000])
plt.grid()
plt.show()

In [ ]:
resampled = resample_poly(phase, 1, 777)
bits = np.hstack((np.diff(resampled)>1, [0]))
plt.plot(bits)
plt.show()
bits.shape, 107*97


In [ ]:
plt.imshow(bits.reshape(107,97), cmap='gray')

# Trash AM Demod Down Here
At one point I encoded the signal as a grayscale image, but thought black and white was easier to detect.

In [ ]:
with Image.open('2022-09_gort-wanted_v1.png') as img:
    # display(img)
    # display(img.convert('1'))
    npimg = np.array(img.convert('L'))
plt.imshow(npimg)
plt.show()

In [ ]:
am_bits = npimg.ravel().astype(np.float).repeat(sps)
am_bits.shape

In [ ]:
baud = 2000
samp_rate = 400e3
print(10379/baud)
sps = samp_rate/baud
print('sps',sps)

In [ ]:
am_bits

In [ ]:
# shiftray = np.exp(1j*np.arange(len(am_bits))) * 2 * np.pi *  / samp_rate
shifted = swiftfox.freqshift((am_bits.astype(np.complex64) / 3e6), samp_rate, -75e3)
merged = shifted + samps[0:len(shifted)]
shifted.shape, samps.shape

In [ ]:
unshifted = swiftfox.freqshift(merged, samp_rate, 75e3)

In [ ]:
plt.figure(figsize=(16,9))
_=swiftfox.smartspec(unshifted, samp_rate=samp_rate)

In [ ]:
from scipy.signal import resample_poly

In [ ]:
zippy = resample_poly(np.abs(butter_lowpass_filter(unshifted, 5e3, 400e3)), 1, sps)
plt.plot(zippy[0:97*10])
plt.show()

In [ ]:
plt.imshow(zippy.reshape(107,97), cmap='gray')
plt.show()